In [ ]:
import eventlet
import socketio
from flask import Flask
from io import BytesIO
from PIL import Image
import cv2
import base64
import numpy as np
import tensorflow as tf


model = tf.keras.models.load_model('004.h5')
sio = socketio.Server() ## create a socketio
app = Flask(__name__)

speed_limit = 20

@sio.on('connect')
def connect(sid,env):
    print("websocket建立起来了")
    ## websocket has been setup
    
    
@sio.on('telemetry')
def telemetry(sid,data):
    image = Image.open(BytesIO(base64.b64decode(data['image'])))
    image = np.asarray(image)
#    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    cv2.imshow("camera",image)
    cv2.waitKey(10)
    
    ## get the angle according to picture
    image = image[60:135,:,:]
    image = cv2.GaussianBlur(image,(3,3),1)
    image = cv2.resize(image,(200,66))
    image = cv2.cvtColor(image,cv2.COLOR_RGB2YUV)
    
    image  = image/255.0
    angle = model.predict(image.reshape(1,66,200,3))[0][0]
    
    
    ## current speed and throttle
    speed = float(data['speed'])
    throttle = 1.0 - speed/ speed_limit
    
    ## send angle to simulator server
    sio.emit("steer",data={
        'steering_angle':(angle-0.085).__str__(),
        'throttle': throttle.__str__()
    })
    

if __name__=='__main__':
    # 把connenct local web and socketio
    app = socketio.Middleware(sio,app)
    # monitor socket
    eventlet.wsgi.server(eventlet.listen(('',4567)),app)

(16816) wsgi starting up on http://0.0.0.0:4567
(16816) accepted ('127.0.0.1', 62807)


websocket建立起来了


127.0.0.1 - - [17/Jan/2022 18:51:00] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 47.777541
